In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2025-09-20 04:05:53.195399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758341153.471479      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758341153.550801      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# 📂 Dataset paths
annotation_dir = "/kaggle/input/road-sign-detection/annotations"
image_dir = "/kaggle/input/road-sign-detection/images"

# Function to parse a single XML file
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find("filename").text
    objects = []

    for obj in root.findall("object"):
        label = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        objects.append((label, xmin, ymin, xmax, ymax))

    return filename, objects

In [3]:
# 📥 Load images & labels
images = []
labels = []
target_size = (64, 64)  # Resize all crops to this size

for xml_file in os.listdir(annotation_dir):
    if xml_file.endswith(".xml"):
        file_path = os.path.join(annotation_dir, xml_file)
        filename, objects = parse_annotation(file_path)

        img_path = os.path.join(image_dir, filename)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            for label, xmin, ymin, xmax, ymax in objects:
                cropped = img[ymin:ymax, xmin:xmax]
                cropped = cv2.resize(cropped, target_size)
                cropped = cropped / 255.0  # Normalize
                images.append(cropped)
                labels.append(label)

In [4]:
images = np.array(images, dtype=np.float32)
labels = np.array(labels)

In [5]:
# 🔤 Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)


In [6]:
# 📊 Split into train & validation sets
X_train, X_val, y_train, y_val = train_test_split(
    images, labels_categorical, test_size=0.2, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Classes:", label_encoder.classes_)

Train: (995, 64, 64, 3) (995, 4)
Validation: (249, 64, 64, 3) (249, 4)
Classes: ['crosswalk' 'speedlimit' 'stop' 'trafficlight']


In [7]:

# 🧠 CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# ⚙️ Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-20 04:06:37.831537: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,716 (2.61 MB)

 Trainable params: 683,716 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# 🚀 Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32
)

Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 135ms/step - accuracy: 0.6353 - loss: 0.8832 - val_accuracy: 0.8916 - val_loss: 0.2954
Epoch 2/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.8988 - loss: 0.3028 - val_accuracy: 0.9398 - val_loss: 0.1626
Epoch 3/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - accuracy: 0.9364 - loss: 0.1661 - val_accuracy: 0.9639 - val_loss: 0.0910
Epoch 4/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - accuracy: 0.9698 - loss: 0.1018 - val_accuracy: 0.9880 - val_loss: 0.0549
Epoch 5/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.9686 - loss: 0.0845 - val_accuracy: 0.9759 - val_loss: 0.0788
Epoch 6/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - accuracy: 0.9823 - loss: 0.0607 - val_accuracy: 0.9880 - val_loss: 0.0323
Epoch 7/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.9880 - loss: 0.0449 - val_accuracy: 0.9960 - val_loss: 0.0164
Epoch 8/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.9939 - loss: 0.0175 - val_accuracy: 0.

In [9]:
# 💾 Save model & label encoder
model.save("traffic_sign_cnn.h5")

import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)